In [7]:
%run '/Users/christianlangridge/Desktop/Zhang-Lab/Zhang Lab Code/Boilerplate/model_boilerplate.py'

In [10]:
#Loading models back in 

model_dir = '/Users/christianlangridge/Desktop/Zhang-Lab/Zhang Lab Data/Saved models/Random Forest/Saved_Models_XGBRF_v2'

# Find all saved models; ensure consistent order
model_paths = sorted(glob.glob(os.path.join(model_dir, "target_*.json")))

models = []
for path in model_paths:
    est = xgb.XGBRFRegressor(
        objective='reg:squarederror',
        random_state=42,
        n_estimators=100,
        max_depth=5,
        device='cuda',
        tree_method='hist'
    )
    est.load_model(path)
    models.append(est)

print(f"Loaded {len(models)} models from {model_dir}")


Loaded 16101 models from /Users/christianlangridge/Desktop/Zhang-Lab/Zhang Lab Data/Saved models/Random Forest/Saved_Models_XGBRF_v2


In [ ]:
# Evaluate model 
#Loaded object is a list of models. n_models: 16101
#y_pred.shape: (3187, 16101)
#model.n_features_in_: None
#model.n_outputs_: None
#x_test.shape: (3187, 1198)
#y_test.shape: (3187, 16101)
#Multi-output R^2 (uniform_average): -7.29502791702064
#first estimator type: <class 'xgboost.sklearn.XGBRFRegressor'>
#n_features_in_: 1198
#example feature_importances_ (first 10): [2.8079670e-02 0.0000000e+00 1.3237547e-05 2.5622896e-04 0.0000000e+00
# 0.0000000e+00 2.1372591e-04 0.0000000e+00 1.4226705e-05 0.0000000e+00]

# terrible performance here, going to understand source of issue

loaded = models  

if isinstance(loaded, list):
    models = loaded
    print("Loaded object is a list of models. n_models:", len(models))
    y_pred = np.column_stack([m.predict(x_test) for m in models])  # (n_samples, n_genes)
else:
    model = loaded
    print("Loaded object type:", type(model))
    # If single-output model but y_test is multi-column, this will raise -- handled later
    y_pred = model.predict(x_test)

print("y_pred.shape:", y_pred.shape)

# diagnostics (safe access)
def safe_attr(obj, name):
    return getattr(obj, name, None) if not isinstance(obj, list) else None

print("model.n_features_in_:", safe_attr(loaded, "n_features_in_"))
print("model.n_outputs_:", safe_attr(loaded, "n_outputs_"))
print("x_test.shape:", x_test.shape)
print("y_test.shape:", y_test.shape)

# Continue with your R^2 / MSE logic expecting y_pred shape (n_samples, n_targets)
from sklearn.metrics import r2_score, mean_squared_error

# If y_pred is 1D, treat as single-output
if y_pred.ndim == 1 or (y_pred.ndim == 2 and y_pred.shape[1] == 1):
    if y_test.ndim == 2 and y_test.shape[1] > 1:
        raise ValueError(
            "Model predicts a single target but y_test contains multiple targets (genes).\n"
            "Select the trained target column before splitting, e.g.:\n"
            "  target = 'GENE_NAME'\n"
            "  y = gene_expression[target]\n"
            "  then redo train_test_split and evaluation."
        )
    y_true = y_test.ravel()
    print("R^2:", r2_score(y_true, y_pred))
    print("MSE:", mean_squared_error(y_true, y_pred))
else:
    print("Multi-output R^2 (uniform_average):", r2_score(y_test, y_pred, multioutput='uniform_average'))

# For single model or list of models, print info on first estimator
first = models[0] if isinstance(models, list) else models
print("first estimator type:", type(first))
print("n_features_in_:", getattr(first, "n_features_in_", None))
fi = getattr(first, "feature_importances_", None)
if fi is not None:
    print("example feature_importances_ (first 10):", fi[:10])
else:
    print("feature_importances_ not available on first model")

Loaded object is a list of models. n_models: 16101
y_pred.shape: (3187, 16101)
model.n_features_in_: None
model.n_outputs_: None
x_test.shape: (3187, 1198)
y_test.shape: (3187, 16101)
Multi-output R^2 (uniform_average): -7.29502791702064
first estimator type: <class 'xgboost.sklearn.XGBRFRegressor'>
n_features_in_: 1198
example feature_importances_ (first 10): [2.8079670e-02 0.0000000e+00 1.3237547e-05 2.5622896e-04 0.0000000e+00
 0.0000000e+00 2.1372591e-04 0.0000000e+00 1.4226705e-05 0.0000000e+00]


In [ ]:
# checking distribution of r-squared values to confirm if overfitting is the culprit for poor performance 
# validates my hypothesis of overfitting - very negative average score, some very negative individual targets with 
# some more moderately scored ones. Essentially, what I think has happened is an overall overfitting where too many models
# are dialed into the noise of each individual target which is not possible to generalise across novel targets. More over, the previous
# attempt using n-estimators = 3 also isn't great because it introducing underfitting at the "single target level", 
# which I believe has lead to more variability between "harder" fitting genes and "easier" fitting genes in relation to the 
# the multi-output model's ability to generalise on individual target predictions. 

from sklearn.metrics import r2_score

print("Per-target validation R² scores:")
for i, model in enumerate(models):
    val_pred = model.predict(x_val)
    val_r2 = r2_score(y_val[:, i], val_pred)
    print(f"Target {i}: {val_r2:.4f}")

# check aggregate
test_pred = np.column_stack([m.predict(x_test) for m in models])
test_r2 = r2_score(y_test, test_pred)
print(f"\nAggregate test R²: {test_r2:.4f}")

Per-target validation R² scores:
Target 0: 0.9986
Target 1: 0.7983
Target 2: -18.0471
Target 3: -19.5825
Target 4: -11.8880
Target 5: 0.2706
Target 6: 0.2465
Target 7: 0.0067
Target 8: 0.4687
Target 9: 0.5830
Target 10: -0.0668
Target 11: -5.6203
Target 12: -48.5233
Target 13: -5.1390
Target 14: -9.2350
Target 15: -0.8987
Target 16: -1.9623
Target 17: 0.2269
Target 18: -1.8501
Target 19: -0.0414
Target 20: 0.1586
Target 21: -2.4803
Target 22: -1.6208
Target 23: 0.0845
Target 24: -0.0539
Target 25: -16.0680
Target 26: -24.1231
Target 27: -0.3493
Target 28: -1.3948
Target 29: -8.8217
Target 30: -0.9842
Target 31: -1.4855
Target 32: -0.0827
Target 33: -0.2616
Target 34: -1.1069
Target 35: -3.9829
Target 36: -0.5902
Target 37: -1.6522
Target 38: -0.2536
Target 39: -4.5621
Target 40: -1.8952
Target 41: 0.3809
Target 42: -2.6015
Target 43: -3.2674
Target 44: -1.1937
Target 45: -3.0317
Target 46: -2.8125
Target 47: -1.0051
Target 48: -3.0203
Target 49: -2.6208
Target 50: 0.1884
Target 51: -1.